In [1]:
import warnings
import pprint
import skrebate
import imblearn
from imblearn import under_sampling, over_sampling, combine
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn import (preprocessing, svm, linear_model, ensemble, naive_bayes,
                    tree, neighbors, decomposition, kernel_approximation, cluster)
from sklearn.pipeline import Pipeline

from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import (KFold, GroupKFold, StratifiedKFold,
                                    LeaveOneGroupOut, cross_validate,
                                    cross_val_predict, learning_curve,
                                    GridSearchCV)
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel, VarianceThreshold, f_classif
from sklearn.metrics import r2_score, auc, roc_auc_score, balanced_accuracy_score, confusion_matrix
from sklearn.metrics.scorer import roc_auc_scorer
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from xgboost import XGBRegressor, XGBClassifier

warnings.simplefilter('ignore')

/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/Users/guq/galaxy/.venv/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc", line #620
  (fname, cnt))


In [2]:
import numpy as np
import pandas as pd
import re


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [3]:
from iraps_classifier import IRAPSCore, IRAPSClassifier, iraps_auc_scorer

## drug  (5Z)-7-Oxozeaenol

In [25]:
rna = pd.read_csv('./drug_respond/(5Z)-7-Oxozeaenol.csv', sep='\t')
rna

,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),C1orf112 (ENSG00000000460),FGR (ENSG00000000938),CFH (ENSG00000000971),FUCA2 (ENSG00000001036),GCLC (ENSG00000001084),NFYA (ENSG00000001167),...,RP11-309M23.1 (ENSGR0000237531),AMDP1 (ENSGR0000237801),BX649553.1 (ENSGR0000263835),BX649553.2 (ENSGR0000263980),BX649553.3 (ENSGR0000264510),BX649553.4 (ENSGR0000264819),RN7SL355P (ENSGR0000265350),MIR3690 (ENSGR0000265658),AL732314.1 (ENSGR0000266731),(5Z)-7-Oxozeaenol
0,2.650765,0.000000,6.216843,3.427606,4.672991,0.014355,0.111031,5.803744,6.900867,5.287251,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.862564
1,3.001802,0.000000,6.781229,4.150560,3.839960,0.000000,0.298658,7.425510,6.554589,4.295723,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.759749
2,4.590362,0.000000,6.647890,1.899176,3.360364,0.028569,0.536053,5.762615,3.226509,4.024142,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.658579
3,5.881175,0.000000,6.643135,2.039138,5.027243,0.189034,1.575312,6.189232,3.427606,3.980939,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.821017
4,5.045268,0.000000,6.988571,1.883621,3.517276,0.000000,3.795975,6.383704,4.651339,5.055282,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.710453
5,5.943218,0.000000,6.501280,1.937344,4.078951,0.000000,0.432959,5.942280,4.527946,4.472488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.662955
6,0.150560,0.000000,5.460415,2.375735,4.360364,0.042644,0.298658,0.356144,4.182692,5.354382,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.578093
7,3.438293,0.000000,6.313790,2.286881,4.152995,0.000000,1.691534,6.444270,4.759156,4.106013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.662305
8,2.980025,0.000000,7.118630,1.903038,4.533563,0.000000,3.090853,6.827565,3.969933,3.872829,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.699442
9,4.316146,0.000000,6.903400,1.871844,3.569248,0.000000,3.912650,4.234961,3.519793,4.698218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.574803


In [26]:
X, y = rna.iloc[:, :-1], rna.iloc[:, -1]
data1 = go.Scatter(
        y = y,
        mode = "markers",
        name = "y_ture",
)

layout = dict(
    xaxis=dict(
        title='Sample No.'
    ),
    yaxis=dict(
        title='Value'
    ),
    title="Distribution of target y"
    
)
fig = go.Figure(data=[data1], layout=layout)
iplot(fig)


# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

In [6]:
cv = KFold(10, shuffle=True, random_state=0)
scores = []
for train_index, test_index in cv.split(X, y):
    X_train, y_train = X.values[train_index], y[train_index]
    X_test, y_test = X.values[test_index], y[test_index]
    iraps_core = IRAPSCore(n_iter=100, verbose=0, random_state=1)
    iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
    iraps.fit(X_train, y_train)
    y_p = iraps.predict(X_test)
    y_t = y_test < iraps.discretize_value
    auc = roc_auc_score(y_t, y_p)
    print("AUC: %f" %auc)
    scores.append(auc)
print("Mean: %f" %(sum(scores)/float(len(scores))))

AUC: 0.882075
AUC: 0.862981
AUC: 0.782931
AUC: 0.714000
AUC: 0.865229
AUC: 0.792727
AUC: 0.625205
AUC: 0.820000
AUC: 0.762527
AUC: 0.628032
Mean: 0.773571


In [7]:
## gridsearchcv

cv = KFold(10, shuffle=True, random_state=0)
iraps_core = IRAPSCore(n_iter=100, verbose=0, random_state=1)
iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
pip = Pipeline([('estimator', iraps)])
search = GridSearchCV(pip, {}, cv = cv, scoring=iraps_auc_scorer, refit=False, verbose=1, return_train_score=True)

search.fit(X, y)
search.cv_results_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  9.0min finished


{'mean_fit_time': array([54.01811125]),
 'mean_score_time': array([0.01005144]),
 'mean_test_score': array([0.77375135]),
 'mean_train_score': array([0.78825424]),
 'params': [{}],
 'rank_test_score': array([1], dtype=int32),
 'split0_test_score': array([0.88207547]),
 'split0_train_score': array([0.78141304]),
 'split1_test_score': array([0.86298077]),
 'split1_train_score': array([0.78909978]),
 'split2_test_score': array([0.78293135]),
 'split2_train_score': array([0.79273959]),
 'split3_test_score': array([0.714]),
 'split3_train_score': array([0.79589633]),
 'split4_test_score': array([0.86522911]),
 'split4_train_score': array([0.78639291]),
 'split5_test_score': array([0.79272727]),
 'split5_train_score': array([0.78592098]),
 'split6_test_score': array([0.62520458]),
 'split6_train_score': array([0.80458404]),
 'split7_test_score': array([0.82]),
 'split7_train_score': array([0.76895691]),
 'split8_test_score': array([0.76252723]),
 'split8_train_score': array([0.7730836]),
 's

In [27]:
## cross_val_predic

cv = KFold(10, shuffle=True, random_state=0)
iraps_core = IRAPSCore(n_iter=100, positive_thres=-1, verbose=0, random_state=1)
iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
pip = Pipeline([('estimator', iraps)])
result = cross_val_predict(pip, X, y, cv = cv)
result

array([-0.28282882, -0.51479043,  0.71297184, -0.0530746 ,  0.5492146 ,
       -0.32582344,  0.5494661 ,  0.3160137 ,  0.46020004,  0.73579713,
        0.58991713,  0.51857111,  0.87207581,  0.63220428,  0.63641758,
       -0.24283515,  0.68141452,  0.84246806,  0.3289527 ,  0.2910635 ,
        0.18498409,  0.41109521,  0.08138383,  0.47521067,  0.26924621,
       -0.40782592,  0.21398407,  0.47675831,  0.75661665,  0.42380242,
        0.54606795, -0.15713419,  0.43126929, -0.42269826, -0.30432487,
        0.65453633,  0.20114492, -0.21027537, -0.37194531, -0.37774756,
        0.18734586, -0.48817715, -0.54291155, -0.55996895,  0.59757495,
        0.60262401, -0.42953103, -0.33385137,  0.85451034,  0.51188346,
        0.19193411,  0.33588814,  0.43406274,  0.36734997, -0.25089492,
       -0.42153244, -0.45836201, -0.35021055,  0.04632459,  0.16232562,
        0.43027042,  0.62313138, -0.4269332 , -0.43940306,  0.60791721,
       -0.51705865, -0.43383921, -0.37529711,  0.83213445, -0.13

In [31]:
data1 = go.Scatter(
        x = y,
        y = [x if x<1 else 1 for x in (1-result) ],
        mode = "markers",
        name = "True vs. Predicted",
)

data2 = go.Scatter(
        x = [0, 1],
        y = [0, 1],
        mode = "lines",
        name = "Perfect",
)

layout = dict(
    xaxis=dict(
        title='Target'
    ),
    yaxis=dict(
        title='COR to minority'
    ),
    title="Cross val predict"
    
)
fig = go.Figure(data=[data1, data2], layout=layout)
iplot(fig)

# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

## drug  Olaparib_(1)

In [8]:
rna = pd.read_csv('./drug_respond/Olaparib_(1).tsv', sep='\t')
rna

,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),C1orf112 (ENSG00000000460),FGR (ENSG00000000938),CFH (ENSG00000000971),FUCA2 (ENSG00000001036),GCLC (ENSG00000001084),NFYA (ENSG00000001167),...,RP4-671G15.2 (ENSG00000273483),OR6R2P (ENSG00000273484),RP11-225H22.7 (ENSG00000273485),RP11-731C17.2 (ENSG00000273486),RP4-621B10.8 (ENSG00000273487),RP11-114I8.4 (ENSG00000273488),RP11-180C16.1 (ENSG00000273489),AP000230.1 (ENSG00000273492),RP11-80H18.4 (ENSG00000273493),Olaparib (1)
0,2.650765,0.000000,6.216843,3.427606,4.672991,0.014355,0.111031,5.803744,6.900867,5.287251,...,0.910733,0.0,0.443607,1.555816,0.250962,2.432959,3.174726,0.014355,0.000000,0.978863
1,3.001802,0.000000,6.781229,4.150560,3.839960,0.000000,0.298658,7.425510,6.554589,4.295723,...,0.356144,0.0,0.000000,0.748461,0.070389,1.570463,3.065228,0.111031,0.000000,0.985041
2,4.590362,0.000000,6.647890,1.899176,3.360364,0.028569,0.536053,5.762615,3.226509,4.024142,...,0.150560,0.0,0.565597,0.432959,0.000000,0.815575,1.454176,0.014355,0.000000,0.922960
3,5.881175,0.000000,6.643135,2.039138,5.027243,0.189034,1.575312,6.189232,3.427606,3.980939,...,0.000000,0.0,0.000000,0.176323,0.097611,0.475085,2.049631,0.084064,0.000000,0.915580
4,5.045268,0.000000,6.988571,1.883621,3.517276,0.000000,3.795975,6.383704,4.651339,5.055282,...,0.124328,0.0,0.263034,0.584963,0.000000,1.570463,1.970854,0.150560,0.000000,0.938053
5,5.943218,0.000000,6.501280,1.937344,4.078951,0.000000,0.432959,5.942280,4.527946,4.472488,...,0.124328,0.0,0.000000,0.344828,0.028569,0.411426,1.244887,0.000000,0.000000,0.978103
6,0.150560,0.000000,5.460415,2.375735,4.360364,0.042644,0.298658,0.356144,4.182692,5.354382,...,0.000000,0.0,0.214125,0.963474,0.000000,1.636915,1.978196,0.084064,1.495695,0.824990
7,4.316146,0.000000,6.903400,1.871844,3.569248,0.000000,3.912650,4.234961,3.519793,4.698218,...,0.275007,0.0,0.176323,0.695994,0.000000,0.799087,1.655352,0.000000,0.000000,0.973685
8,4.724650,0.000000,6.957450,1.835924,3.682573,0.000000,0.454176,5.825277,3.918386,4.197708,...,0.238787,0.0,0.879706,0.887525,0.000000,1.070389,1.495695,0.000000,0.000000,0.925170
9,5.108106,0.000000,6.888500,2.207893,4.008092,0.505891,2.611172,6.316146,5.361417,4.190615,...,0.495695,0.0,0.641546,1.344828,0.189034,1.839960,1.887525,0.163499,0.000000,0.901699


In [9]:
X, y = rna.iloc[:, :-1], rna.iloc[:, -1]
data1 = go.Scatter(
        y = y,
        mode = "markers",
        name = "y_ture",
)

layout = dict(
    xaxis=dict(
        title='Sample No.'
    ),
    yaxis=dict(
        title='Value'
    ),
    title="Distribution of target y"
    
)
fig = go.Figure(data=[data1], layout=layout)
iplot(fig)

# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

In [10]:
cv = KFold(10, shuffle=True, random_state=0)
scores = []
for train_index, test_index in cv.split(X, y):
    X_train, y_train = X.values[train_index], y[train_index]
    X_test, y_test = X.values[test_index], y[test_index]
    iraps_core = IRAPSCore(n_iter=100, verbose=0, random_state=1)
    iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
    iraps.fit(X_train, y_train)
    y_p = iraps.predict(X_test)
    y_t = y_test < iraps.discretize_value
    auc = roc_auc_score(y_t, y_p)
    print("AUC: %f" %auc)
    scores.append(auc)
print("Mean: %f" %(sum(scores)/float(len(scores))))

AUC: 0.687500
AUC: 0.830189
AUC: 0.725490
AUC: 0.700680
AUC: 0.897436
AUC: 0.382353
AUC: 0.741497
AUC: 0.613095
AUC: 0.676000
AUC: 0.688000
Mean: 0.694224


In [11]:
## gridsearchcv

cv = KFold(10, shuffle=True, random_state=0)
iraps_core = IRAPSCore(n_iter=100, verbose=0, random_state=1)
iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
pip = Pipeline([('estimator', iraps)])
search = GridSearchCV(pip, {}, cv = cv, scoring=iraps_auc_scorer, refit=False, verbose=1, return_train_score=True)

search.fit(X, y)
search.cv_results_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  7.0min finished


{'mean_fit_time': array([41.83205514]),
 'mean_score_time': array([0.00880823]),
 'mean_test_score': array([0.69421178]),
 'mean_train_score': array([0.79422757]),
 'params': [{}],
 'rank_test_score': array([1], dtype=int32),
 'split0_test_score': array([0.6875]),
 'split0_train_score': array([0.80947125]),
 'split1_test_score': array([0.83018868]),
 'split1_train_score': array([0.77425121]),
 'split2_test_score': array([0.7254902]),
 'split2_train_score': array([0.79945799]),
 'split3_test_score': array([0.70068027]),
 'split3_train_score': array([0.8040527]),
 'split4_test_score': array([0.8974359]),
 'split4_train_score': array([0.7902194]),
 'split5_test_score': array([0.38235294]),
 'split5_train_score': array([0.81670852]),
 'split6_test_score': array([0.7414966]),
 'split6_train_score': array([0.74813836]),
 'split7_test_score': array([0.61309524]),
 'split7_train_score': array([0.78338164]),
 'split8_test_score': array([0.676]),
 'split8_train_score': array([0.81260056]),
 'spl

## test get_params

In [12]:
iraps_core = IRAPSCore(n_iter=100, verbose=0, random_state=1)
iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
pip = Pipeline([('estimator', iraps)])
iraps.get_params()

{'discretize': 'z_score',
 'fc_thres': 0.1,
 'iraps_core': IRAPSCore(n_iter=100, negative_thres=0, positive_thres=-1, random_state=1,
      verbose=0),
 'iraps_core__n_iter': 100,
 'iraps_core__negative_thres': 0,
 'iraps_core__positive_thres': -1,
 'iraps_core__random_state': 1,
 'iraps_core__verbose': 0,
 'occurance': 0.7,
 'p_thres': 0.01}

In [ ]:
## drug Sepantronium bromide -- a reverse example

In [13]:
rna = pd.read_csv('./Galaxy216-[Depmap_drug_rna-seq_ready__Sepantronium bromide.csv].tabular', sep='\t')
rna

,TSPAN6 (ENSG00000000003),TNMD (ENSG00000000005),DPM1 (ENSG00000000419),SCYL3 (ENSG00000000457),C1orf112 (ENSG00000000460),FGR (ENSG00000000938),CFH (ENSG00000000971),FUCA2 (ENSG00000001036),GCLC (ENSG00000001084),NFYA (ENSG00000001167),...,RP4-671G15.2 (ENSG00000273483),OR6R2P (ENSG00000273484),RP11-225H22.7 (ENSG00000273485),RP11-731C17.2 (ENSG00000273486),RP4-621B10.8 (ENSG00000273487),RP11-114I8.4 (ENSG00000273488),RP11-180C16.1 (ENSG00000273489),AP000230.1 (ENSG00000273492),RP11-80H18.4 (ENSG00000273493),Sepantronium bromide
0,2.650765,0.000000,6.216843,3.427606,4.672991,0.014355,0.111031,5.803744,6.900867,5.287251,...,0.910733,0.0,0.443607,1.555816,0.250962,2.432959,3.174726,0.014355,0.000000,0.069809
1,3.001802,0.000000,6.781229,4.150560,3.839960,0.000000,0.298658,7.425510,6.554589,4.295723,...,0.356144,0.0,0.000000,0.748461,0.070389,1.570463,3.065228,0.111031,0.000000,0.211675
2,4.590362,0.000000,6.647890,1.899176,3.360364,0.028569,0.536053,5.762615,3.226509,4.024142,...,0.150560,0.0,0.565597,0.432959,0.000000,0.815575,1.454176,0.014355,0.000000,0.029108
3,5.881175,0.000000,6.643135,2.039138,5.027243,0.189034,1.575312,6.189232,3.427606,3.980939,...,0.000000,0.0,0.000000,0.176323,0.097611,0.475085,2.049631,0.084064,0.000000,0.022355
4,5.045268,0.000000,6.988571,1.883621,3.517276,0.000000,3.795975,6.383704,4.651339,5.055282,...,0.124328,0.0,0.263034,0.584963,0.000000,1.570463,1.970854,0.150560,0.000000,0.035696
5,5.943218,0.000000,6.501280,1.937344,4.078951,0.000000,0.432959,5.942280,4.527946,4.472488,...,0.124328,0.0,0.000000,0.344828,0.028569,0.411426,1.244887,0.000000,0.000000,0.094998
6,0.150560,0.000000,5.460415,2.375735,4.360364,0.042644,0.298658,0.356144,4.182692,5.354382,...,0.000000,0.0,0.214125,0.963474,0.000000,1.636915,1.978196,0.084064,1.495695,0.012873
7,3.438293,0.000000,6.313790,2.286881,4.152995,0.000000,1.691534,6.444270,4.759156,4.106013,...,0.124328,0.0,0.356144,0.389567,0.084064,1.400538,2.179511,0.201634,0.000000,0.030631
8,2.980025,0.000000,7.118630,1.903038,4.533563,0.000000,3.090853,6.827565,3.969933,3.872829,...,0.250962,0.0,0.176323,0.963474,0.000000,1.516015,3.451541,0.111031,0.000000,0.021333
9,4.316146,0.000000,6.903400,1.871844,3.569248,0.000000,3.912650,4.234961,3.519793,4.698218,...,0.275007,0.0,0.176323,0.695994,0.000000,0.799087,1.655352,0.000000,0.000000,0.974812


In [14]:
X, y = rna.iloc[:, :-1], rna.iloc[:, -1]
data1 = go.Scatter(
        y = y,
        mode = "markers",
        name = "y_ture",
)

layout = dict(
    xaxis=dict(
        title='Sample No.'
    ),
    yaxis=dict(
        title='Value'
    ),
    title="Distribution of target y"
    
)
fig = go.Figure(data=[data1], layout=layout)
iplot(fig)

# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/

In [15]:
cv = KFold(10, shuffle=True, random_state=0)
scores = []
for train_index, test_index in cv.split(X, y):
    X_train, y_train = X.values[train_index], y[train_index]
    X_test, y_test = X.values[test_index], y[test_index]
    iraps_core = IRAPSCore(n_iter=100, positive_thres=1, verbose=0, random_state=1)
    iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
    iraps.fit(X_train, y_train)
    y_p = iraps.predict(X_test)
    y_t = y_test > iraps.discretize_value
    auc = roc_auc_score(y_t, y_p)
    print("AUC: %f" %auc)
    scores.append(auc)
print("Mean: %f" %(sum(scores)/float(len(scores))))

AUC: 0.719697
AUC: 0.508242
AUC: 0.702222
AUC: 0.760000
AUC: 0.845588
AUC: 0.880000
AUC: 0.617021
AUC: 0.672348
AUC: 0.758242
AUC: 0.606383
Mean: 0.706974


In [16]:
## gridsearchcv

cv = KFold(10, shuffle=True, random_state=0)
iraps_core = IRAPSCore(n_iter=100, positive_thres=1, verbose=0, random_state=1)
iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
pip = Pipeline([('estimator', iraps)])
search = GridSearchCV(pip, {}, cv = cv, scoring=iraps_auc_scorer, refit=False, verbose=1, return_train_score=True)

search.fit(X, y)
search.cv_results_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  7.7min finished


{'mean_fit_time': array([45.98017857]),
 'mean_score_time': array([0.00831873]),
 'mean_test_score': array([0.70697437]),
 'mean_train_score': array([0.78283272]),
 'params': [{}],
 'rank_test_score': array([1], dtype=int32),
 'split0_test_score': array([0.71969697]),
 'split0_train_score': array([0.78963446]),
 'split1_test_score': array([0.50824176]),
 'split1_train_score': array([0.79142305]),
 'split2_test_score': array([0.70222222]),
 'split2_train_score': array([0.78748284]),
 'split3_test_score': array([0.76]),
 'split3_train_score': array([0.78926229]),
 'split4_test_score': array([0.84558824]),
 'split4_train_score': array([0.77301587]),
 'split5_test_score': array([0.88]),
 'split5_train_score': array([0.77754334]),
 'split6_test_score': array([0.61702128]),
 'split6_train_score': array([0.7790204]),
 'split7_test_score': array([0.67234848]),
 'split7_train_score': array([0.80493424]),
 'split8_test_score': array([0.75824176]),
 'split8_train_score': array([0.75469012]),
 'sp

In [17]:
## cross_val_predic

cv = KFold(10, shuffle=True, random_state=0)
iraps_core = IRAPSCore(n_iter=100, positive_thres=1, verbose=0, random_state=1)
iraps = IRAPSClassifier(iraps_core, p_thres=0.01, occurance=0.7)
pip = Pipeline([('estimator', iraps)])
result = cross_val_predict(pip, X, y, cv = cv)
result

array([ 0.51546199,  0.75287796,  0.2602504 ,  0.10791174,  0.11455634,
        0.0122205 , -0.54210161,  0.76118462,  0.44097194, -0.13221415,
        0.12380421,  0.07119612,  0.45361211,  0.36775752,  0.11641102,
        0.36117464,  0.21942322,  0.23640962,  0.44675613, -0.35703672,
        0.37972038,  0.54999164, -0.37998566,  0.06183197,  0.45935884,
        0.67583603,  0.72470275, -0.67991489,  0.10027925, -0.47312613,
        0.48740836,  0.76611682,  0.09569631,  0.23568334,  0.64114438,
        0.32086423, -0.02301382,  0.3131974 , -0.09379264,  0.1482719 ,
        0.01452258, -0.64761653,  0.16905015,  0.27040797,  0.46030464,
       -0.58539995,  0.25958791,  0.74644248,  0.44566126,  0.26768992,
       -0.68382298,  0.41204668,  0.45963051,  0.02169097, -0.17905746,
        0.36402064,  0.1383893 ,  0.31115877, -0.00286284,  0.65309787,
        0.71952337,  0.16970409,  0.38944263,  0.09689484,  0.65581319,
        0.38330792,  0.41258879,  0.22959972,  0.49380089,  0.43

In [24]:
data1 = go.Scatter(
        x = y,
        y = result,
        mode = "markers",
        name = "True vs. Predicted",
)

data2 = go.Scatter(
        x = [0, 1],
        y = [0, 1],
        mode = "lines",
        name = "Perfect",
)

layout = dict(
    xaxis=dict(
        title='Target'
    ),
    yaxis=dict(
        title='COR to minority'
    ),
    title="Cross val predict"
    
)
fig = go.Figure(data=[data1, data2], layout=layout)
iplot(fig)

# To show plot, paste the link to this GitHub notebook into http://nbviewer.jupyter.org/